# Extração de dados

- passos
    1. Com base mas colunas da tabela do censo de 2021 de curso e IES será uma criada uma tabela para cursos superiores e institutos de ensino superior, contendo as informações pertinentes para a analise ao longo dos anos dos cursos de formação superior.
    2. A tabela de cursos superiores e institutos de ensino superior e serão normalizadas e carregadas dentro de um banco de dados PostegreeSQL.
    3. As cargas serão verificadas de forma quantitativa para verificar a veraicidade dos dados.

In [35]:
import pandas as pd
import numpy as np

In [36]:
cursos = pd.DataFrame
ies = pd.DataFrame

In [37]:
print('lendo as tabelas...')
for ano in range(21, 8, -1):
    for tabela in ['CURSOS', 'IES']:
        if ano == 9:
            ano = '09'
        
        exec(f"df = pd.read_csv('data/20{ano}/MICRODADOS_CADASTRO_{tabela}_20{ano}.csv', sep=';', encoding='latin1', low_memory=False)")

        _tabela = tabela.lower()
        if ano == 21:
            exec(f'{_tabela} = df')
        else:
            exec(f'{_tabela} = pd.concat([{_tabela}, df], ignore_index=True)')

    print(f'data 20{ano} : OK.')
del(df)

lendo as tabelas...
data 2021 : OK.
data 2020 : OK.
data 2019 : OK.
data 2018 : OK.
data 2017 : OK.
data 2016 : OK.
data 2015 : OK.
data 2014 : OK.
data 2013 : OK.
data 2012 : OK.
data 2011 : OK.
data 2010 : OK.
data 2009 : OK.


In [ ]:
#export ies and cursos
# ies.to_csv('data/ies.csv', index=False)
# cursos.to_csv('data/cursos.csv', index=False)

# Transformação de Dados

In [4]:
df_cursos = cursos
cursos = cursos[['NU_ANO_CENSO', 'CO_CURSO', 'CO_IES', 'NO_CURSO', 'TP_GRAU_ACADEMICO', 'CO_CINE_AREA_DETALHADA']].reset_index(drop=True)

In [5]:
# criar uma tabela com o ano que o co_curso aparece pela primeira vez
cursos['ANO_INICIO'] = cursos.groupby('CO_CURSO')['NU_ANO_CENSO'].transform('min')
# criar uma tabela com o ano que o co_curso aparece pela ultima vez
cursos['ANO_FIM'] = cursos.groupby('CO_CURSO')['NU_ANO_CENSO'].transform('max')

In [6]:
# deletar linhas duplicadas por co_curso e retirar a coluna NU_ANO_CENSO e resetar o index
cursos = cursos.drop_duplicates(subset=['CO_CURSO']).drop(columns=['NU_ANO_CENSO']).reset_index(drop=True)

In [7]:
df_ies = ies
ies = ies[['CO_IES', 'NO_IES', 'SG_IES', 'CO_REGIAO_IES', 'CO_UF_IES', 'CO_MUNICIPIO_IES']].drop_duplicates(subset=['CO_IES']).reset_index(drop=True)

In [8]:
regiao = df_ies[['CO_REGIAO_IES', 'NO_REGIAO_IES']].drop_duplicates(subset=['CO_REGIAO_IES']).sort_values(by='CO_REGIAO_IES').reset_index(drop=True)

In [9]:
uf = df_ies[['CO_UF_IES', 'SG_UF_IES', 'NO_UF_IES']].drop_duplicates(subset=['CO_UF_IES']).sort_values(by='CO_UF_IES').reset_index(drop=True)

In [10]:
municipios = df_ies[['CO_MUNICIPIO_IES', 'NO_MUNICIPIO_IES', 'CO_UF_IES']].drop_duplicates(subset=['CO_MUNICIPIO_IES']).sort_values(by='CO_MUNICIPIO_IES').reset_index(drop=True)

In [12]:
qts_alunos = df_cursos[['CO_CURSO', 'NU_ANO_CENSO', 'QT_VG_TOTAL', 'QT_VG_NOVA', 'QT_INSCRITO_TOTAL', 'QT_ING', 'QT_MAT', 'QT_CONC']]
#drop duplicates por oc_curso e nu_ano_censo
qts_alunos = qts_alunos.drop_duplicates(subset=['CO_CURSO', 'NU_ANO_CENSO']).reset_index(drop=True)

In [13]:
cine_area_detalhada = df_cursos[['CO_CINE_AREA_DETALHADA', 'NO_CINE_AREA_DETALHADA']].drop_duplicates(subset=['CO_CINE_AREA_DETALHADA']).sort_values(by='CO_CINE_AREA_DETALHADA').reset_index(drop=True)

# Carga para o bando de dados

In [14]:
import psycopg2

In [15]:
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    port=5432,
    user="postgres",
    password="99885")

In [16]:
cursor = conn.cursor()

In [19]:
try:
    cursor.execute("CREATE SCHEMA censo_ensino_superior;")
    conn.commit()
except:
    print('schema ja existe')
    conn.commit()

schema ja existe


### Cine Area

In [20]:
# criar tebela cine area
cursor.execute("""
    CREATE TABLE IF NOT EXISTS censo_ensino_superior.cine_area_detalhada (
        CO_CINE_AREA_DETALHADA INT PRIMARY KEY,
        NO_CINE_AREA_DETALHADA VARCHAR(255)
    );
""")
conn.commit()


In [ ]:
#carregar tabela cine area
for i in range(len(cine_area_detalhada)):
    cursor.execute(f"INSERT INTO censo_ensino_superior.cine_area_detalhada VALUES ({cine_area_detalhada.loc[i, 'CO_CINE_AREA_DETALHADA']}, '{cine_area_detalhada.loc[i, 'NO_CINE_AREA_DETALHADA']}');")
conn.commit()

### Regiao

In [ ]:
# criar tabela regiao

cursor.execute("""
    CREATE TABLE IF NOT EXISTS censo_ensino_superior.regiao (
        co_regiao_ies int PRIMARY KEY,
        no_regiao_ies varchar(50)
    );
""")
conn.commit()




In [ ]:
# carregar tabela regiao
try:
    for i in range(len(regiao)):
        cursor.execute(f"""
            INSERT INTO censo_ensino_superior.regiao (co_regiao_ies, no_regiao_ies)
            VALUES ({regiao['CO_REGIAO_IES'][i]}, '{regiao['NO_REGIAO_IES'][i]}');
        """)
    conn.commit()
except:
    print('tabela regiao ja foi carregada')

tabela regiao ja foi carregada


### UF

In [ ]:
#criar tabela uf no banco

cursor.execute("""
    CREATE TABLE IF NOT EXISTS censo_ensino_superior.uf (
        co_uf_ies int PRIMARY KEY,
        sg_uf_ies varchar(2),
        no_uf_ies varchar(50)
    );
""")
conn.commit()


tabela uf ja existe


In [ ]:
# carregar tabela uf
for i in range(len(uf)):
    cursor.execute(f"""
        INSERT INTO censo_ensino_superior.uf (co_uf_ies, sg_uf_ies, no_uf_ies)
        VALUES ({uf['CO_UF_IES'][i]}, '{uf['SG_UF_IES'][i]}', '{uf['NO_UF_IES'][i]}');
    """)
    if i % 1000 == 0:
        conn.commit()
        print(i)
conn.commit()

tabela uf ja foi carregada


### MUNICIPIO

In [ ]:
# criar tabela municipio

cursor.execute("""
    CREATE TABLE IF NOT EXISTS censo_ensino_superior.municipio (
        co_municipio_ies int PRIMARY KEY,
        no_municipio_ies varchar(50),
        co_uf_ies int,
        
        FOREIGN KEY (co_uf_ies) REFERENCES censo_ensino_superior.uf(co_uf_ies)
    );
""")
conn.commit()


In [ ]:
# mudar todos d'Oeste para d.Oeste
municipios['NO_MUNICIPIO_IES'] = municipios['NO_MUNICIPIO_IES'].str.replace("d'Oeste", "d.Oeste")

In [ ]:
#carregar tabela municipio
for i in range(len(municipios)):
    cursor.execute(f"""
        INSERT INTO censo_ensino_superior.municipio (co_municipio_ies, no_municipio_ies, co_uf_ies)
        VALUES ({municipios['CO_MUNICIPIO_IES'][i]}, '{municipios['NO_MUNICIPIO_IES'][i]}', {municipios['CO_UF_IES'][i]});
    """)
if i % 1000 == 0:
    conn.commit()
    print(i)
conn.commit()

### IES

In [ ]:
# tabela ies
cursor.execute("""
    CREATE TABLE IF NOT EXISTS censo_ensino_superior.ies (
        co_ies int PRIMARY KEY,
        no_ies varchar(255),
        sg_ies varchar(50),
        co_municipio_ies int,
        co_regiao_ies int,

        FOREIGN KEY (co_municipio_ies) REFERENCES censo_ensino_superior.municipio(co_municipio_ies),
        FOREIGN KEY (co_regiao_ies) REFERENCES censo_ensino_superior.regiao(co_regiao_ies)
    );
""")
conn.commit()

In [ ]:
#subistituir as "'" por "."
ies['NO_IES'] = ies['NO_IES'].str.replace("'", ".")
ies['SG_IES'] = ies['SG_IES'].str.replace("'", ".")


In [ ]:
# carregar ies
for i in range(len(ies)):
    cursor.execute(f"""
        INSERT INTO censo_ensino_superior.ies (co_ies, no_ies, sg_ies, co_municipio_ies, co_regiao_ies)
        VALUES ({ies['CO_IES'][i]}, '{ies['NO_IES'][i]}', '{ies['SG_IES'][i]}', {ies['CO_MUNICIPIO_IES'][i]}, {ies['CO_REGIAO_IES'][i]});
    """)
    if i % 1000 == 0:
        conn.commit()
        print(i)
conn.commit()


### CURSO

In [ ]:
# criar tabela cursos
cursor.execute("""
    CREATE TABLE IF NOT EXISTS censo_ensino_superior.cursos (
        co_curso int PRIMARY KEY,
        no_curso varchar(255),
        co_ies int,
        co_cine_area_detalhada int,

        FOREIGN KEY (co_ies) REFERENCES censo_ensino_superior.ies(co_ies),
        FOREIGN KEY (co_cine_area_detalhada) REFERENCES censo_ensino_superior.cine_area_detalhada(co_cine_area_detalhada)
    );
""")
conn.commit()

In [ ]:
# carregar a tabela cursos
for i in range(len(cursos)):
    cursor.execute(f"""
        INSERT INTO censo_ensino_superior.cursos (co_curso, no_curso, co_ies, co_cine_area_detalhada)
        VALUES ({cursos['CO_CURSO'][i]}, '{cursos['NO_CURSO'][i]}', {cursos['CO_IES'][i]}, {cursos['CO_CINE_AREA_DETALHADA'][i]});
    """)
    if i % 1000 == 0:
        conn.commit()
        print(i)
conn.commit()


### QT_Alunos

In [27]:
# criar tabela qt_alunos
cursor.execute("""
    CREATE TABLE IF NOT EXISTS censo_ensino_superior.qt_alunos (
        co_curso int,
        nu_ano_censo int,
        qt_vg_total int,
        qt_vg_nova int,
        qt_incritos_total int,
        qt_ing int,
        qt_mat int,
        qt_conc int,

        PRIMARY KEY (co_curso, nu_ano_censo),
        FOREIGN KEY (co_curso) REFERENCES censo_ensino_superior.cursos(co_curso)

    );
""")
conn.commit()


In [30]:
#substituir nan por not a number numpy
qts_alunos = qts_alunos.replace(np.nan, 0)


In [33]:
# carregar a tabela qt_alunos
for i in range(len(qts_alunos)):
    cursor.execute(f"""
        INSERT INTO censo_ensino_superior.qt_alunos (co_curso, nu_ano_censo, qt_vg_total, qt_vg_nova, qt_incritos_total, qt_ing, qt_mat, qt_conc)
        VALUES ({qts_alunos['CO_CURSO'][i]}, {qts_alunos['NU_ANO_CENSO'][i]}, {qts_alunos['QT_VG_TOTAL'][i]}, {qts_alunos['QT_VG_NOVA'][i]}, {qts_alunos['QT_INSCRITO_TOTAL'][i]}, {qts_alunos['QT_ING'][i]}, {qts_alunos['QT_MAT'][i]}, {qts_alunos['QT_CONC'][i]});
    """)
    if i % 1000 == 0:
        conn.commit()
        print(i)
conn.commit()